In [1]:
# import sys to access script arguments (experiment, ensemble, first_year, last_year)
import sys

In [2]:
# interactive use only
model="ACCESS-ESM1-5"
experiment="historical"

In [3]:
# # Model etc. defined from script input
# model = sys.argv[1]
# print("Model: ", model, " (type: ", type(model), ")")
# experiment = sys.argv[2]
# print("Experiment: ", experiment, " (type: ", type(experiment), ")")

In [4]:
# 1. Load packages

# Import os for makedirs/isfile/environ
import os
os.environ["PYTHONWARNINGS"] = "ignore"

# Load dask
from dask.distributed import Client

# import glob for searching directories
from glob import glob

# Load xarray for N-dimensional arrays
import xarray as xr

# Load traceback to print exceptions
import traceback

# # Load xmip for preprocessing (trying to get consistent metadata for making matrices down the road)
# from xmip.preprocessing import combined_preprocessing

In [5]:
# Create directory on scratch to save the data
scratchdatadir = '/scratch/xv83/TMIP/data'
gdatadatadir = '/g/data/xv83/TMIP/data'

# members = ["HI-05", "HI-06", "HI-07", "HI-08"]
members = ["HI-05", "HI-06"]
# year_start = 1850
year_start = 1980
year_end = 1990

In [6]:
client = Client(n_workers=12)

In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 36,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34789,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 36
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:46211,Total threads: 3
Dashboard: /proxy/35127/status,Memory: 10.43 GiB
Nanny: tcp://127.0.0.1:45579,


In [8]:
# for member in members:
member = members[0]

In [9]:
# print ensemble/member
inputdir = f'/scratch/p66/pbd562/petrichor/get/{member}/history/ocn'
outputdir = f'{gdatadatadir}/{model}/{member}'
print(f"\nProcessing {member}")


Processing HI-05


In [10]:
# directory to save the data to (as NetCDF)
print("Creating directory: ", outputdir)
os.makedirs(outputdir, exist_ok=True)

Creating directory:  /g/data/xv83/TMIP/data/ACCESS-ESM1-5/HI-05


In [11]:
# for decade in range(year_start, year_end, 10):
decade = year_start

print(f'\nDecade {decade}')


Decade 1980


In [12]:
# subset of the files required
# paths = [f'{inputdir}/history/ocn/ocean_month.nc-{year}1231' for year in range(year_start, year_end)]
# paths = [f'{inputdir}/history/ocn/ocean_month.nc-{year}1231' for year in range(1990, 2000)]
if decade == 2010:
    num_years = 5
else:
    num_years = 10

In [13]:
paths = [f'{inputdir}/ocean_month.nc-{year}1231' for year in range(decade, decade + num_years)]

In [24]:
# load the data
ds = xr.open_mfdataset(
    paths,
    chunks={'time':-1, 'st_ocean':-1},
    concat_dim="time",
    compat='override',
    preprocess=None,
    # preprocess=combined_preprocessing,
    engine='netcdf4',
    data_vars='minimal',
    coords='minimal',
    combine='nested',
    parallel=True,
    join='outer',
    attrs_file=None,
    combine_attrs='override',
    drop_variables=[ # Drop all the variables I don't need. Can I use a `keep_variables` instead?
        'ht', 'hu', 'kmt', 'kmu', 'pbot_t', 'patm_t', 'sea_level', 'sea_level_sq',
        'rho', 'rho_dzt', 'dht', 'pot_temp', 'sst', 'sst_sq', 'salt', 'sss', 'pot_rho_0',
        'age_global', 'psiu', 'mld', 'mld_max', 'mld_min', 'mld_sq', 'hblt_max',
        'u', 'v', 'tz_trans', 'tz_trans_sq', 'tx_trans_rho', 'ty_trans_rho',
        'tx_trans_rho_gm', 'ty_trans_rho_gm', 'temp_xflux_ndiffuse_int_z', 'temp_yflux_ndiffuse_int_z',
        'temp_xflux_sigma', 'temp_yflux_sigma', 'temp_yflux_submeso_int_z', 'temp_xflux_submeso_int_z',
        'temp_merid_flux_advect_global', 'temp_merid_flux_over_global', 'temp_merid_flux_gyre_global',
        'salt_merid_flux_advect_global', 'salt_merid_flux_over_global', 'salt_merid_flux_gyre_global',
        'temp_merid_flux_advect_southern', 'temp_merid_flux_over_southern', 'temp_merid_flux_gyre_southern',
        'salt_merid_flux_advect_southern', 'salt_merid_flux_over_southern', 'salt_merid_flux_gyre_southern',
        'temp_merid_flux_advect_atlantic', 'temp_merid_flux_over_atlantic', 'temp_merid_flux_gyre_atlantic',
        'salt_merid_flux_advect_atlantic', 'salt_merid_flux_over_atlantic', 'salt_merid_flux_gyre_atlantic',
        'temp_merid_flux_advect_pacific', 'temp_merid_flux_over_pacific', 'temp_merid_flux_gyre_pacific',
        'salt_merid_flux_advect_pacific', 'salt_merid_flux_over_pacific', 'salt_merid_flux_gyre_pacific',
        'temp_merid_flux_advect_arctic', 'temp_merid_flux_over_arctic', 'temp_merid_flux_gyre_arctic',
        'salt_merid_flux_advect_arctic', 'salt_merid_flux_over_arctic', 'salt_merid_flux_gyre_arctic',
        'temp_merid_flux_advect_indian', 'temp_merid_flux_over_indian', 'temp_merid_flux_gyre_indian',
        'salt_merid_flux_advect_indian', 'salt_merid_flux_over_indian', 'salt_merid_flux_gyre_indian',
        'lprec', 'fprec', 'evap', 'runoff', 'ice_calving', 'melt', 'pme_river', 'sfc_salt_flux_ice',
        'sfc_salt_flux_runoff', 'sfc_hflux_from_water_prec', 'sfc_hflux_from_water_evap',
        'sfc_hflux_from_runoff', 'sfc_hflux_from_calving', 'fprec_melt_heat', 'calving_melt_heat',
        'lw_heat', 'evap_heat', 'sens_heat', 'swflx', 'sw_heat', 'tau_x', 'tau_y', 'frazil_2d',
        'pme', 'pme_mass', 'river', 'swflx_vis', 'sw_frac', 'sfc_hflux_coupler', 'sfc_hflux_pme',
        'tau_curl', 'ekman_we', 'salt_calvingmix', 'temp_calvingmix', 'temp_runoffmix', 'temp_rivermix',
        'temp_runoff', 'temp_calving', 'wfimelt', 'wfiform', 'pbot0', 'anompb', 'eta_t', 'eta_u',
        'rhobarz', 'conv_rho_ud_t', 'urhod', 'vrhod', 'u_surf', 'v_surf', 'u_bott', 'v_bott', 'bottom_temp',
        'bottom_salt', 'bottom_age_global', 'temp', 'temp_xflux_adv', 'temp_yflux_adv', 'temp_zflux_adv',
        'salt_xflux_adv', 'salt_yflux_adv', 'salt_zflux_adv', 'temp_vdiffuse_impl', 'salt_vdiffuse_impl',
        'temp_tendency', 'salt_tendency', 'temp_tendency_expl', 'salt_tendency_expl', 'temp_submeso',
        'salt_submeso', 'neutral_rho', 'pot_rho_2', 'potrho_mix_base', 'potrho_mix_depth', 'press',
        'wt', 'wrhot', 'drhodtheta', 'drhodsalinity', 'cabbeling', 'thermobaricity',
        'salt_xflux_ndiffuse_int_z', 'salt_yflux_ndiffuse_int_z', 'psiv', 'temp_sigma',
        'mixdownslope_temp', 'mixdownslope_salt', 'eddy_depth', 'agm_grid_scaling',
        'bv_freq', 'rossby', 'rossby_radius', 'buoy_freq_ave_submeso', 'hblt_submeso',
        'viscosity_scaling', 'visc_crit_bih', 'lap_fric_u', 'lap_fric_v', 'diff_cbt_wave',
        'diff_cbt_drag', 'bvfreq_bottom', 'mix_efficiency', 'power_waves', 'power_diss_wave',
        'power_diss_drag', 'energy_flux', 'diff_cbt_kpp_t', 'diff_cbt_kpp_s', 'tide_speed_wave',
        'tide_speed_drag', 'tide_speed_mask', 'roughness_length', 'roughness_amp', 'langmuirfactor',
        'bmf_u', 'bmf_v', 'bottom_power_u', 'bottom_power_v', 'wind_power_u', 'wind_power_v',
        'average_T1', 'average_T2', 'average_DT', 'nv', 'potrho', 'potrho_edges'
    ]
)

In [25]:
ds

<xarray.Dataset> Size: 16GB
Dimensions:           (yt_ocean: 300, xt_ocean: 360, yu_ocean: 300,
                       xu_ocean: 360, time: 120, st_ocean: 50, nv: 2,
                       st_edges_ocean: 51, sw_ocean: 50, sw_edges_ocean: 51,
                       grid_xu_ocean: 360, grid_yt_ocean: 300,
                       grid_xt_ocean: 360, grid_yu_ocean: 300)
Coordinates: (12/17)
  * xt_ocean          (xt_ocean) float64 3kB -279.5 -278.5 -277.5 ... 78.5 79.5
  * yt_ocean          (yt_ocean) float64 2kB -77.88 -77.63 ... 89.32 89.77
  * time              (time) datetime64[ns] 960B 1980-01-14T12:00:00 ... 1989...
  * xu_ocean          (xu_ocean) float64 3kB -279.0 -278.0 -277.0 ... 79.0 80.0
  * yu_ocean          (yu_ocean) float64 2kB -77.75 -77.51 -77.26 ... 89.55 90.0
  * st_ocean          (st_ocean) float64 400B 5.0 15.0 ... 5.499e+03 5.831e+03
    ...                ...
  * grid_xt_ocean     (grid_xt_ocean) float64 3kB -279.5 -278.5 ... 78.5 79.5
  * grid_yu_ocean     (grid_yu_ocean) float64 2kB -77.75 -77.51 ... 89.55 90.0
    geolat_c          (yu_ocean, xu_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
    geolon_c          (yu_ocean, xu_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
    geolat_t          (yt_ocean, xt_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
    geolon_t          (yt_ocean, xt_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables:
    area_t            (yt_ocean, xt_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
    area_u            (yu_ocean, xu_ocean) float32 432kB dask.array<chunksize=(300, 360), meta=np.ndarray>
    tx_trans          (time, st_ocean, yt_ocean, xu_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    ty_trans          (time, st_ocean, yu_ocean, xt_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    tx_trans_gm       (time, st_ocean, yt_ocean, xu_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    ty_trans_gm       (time, st_ocean, yu_ocean, xt_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    tx_trans_submeso  (time, st_ocean, yt_ocean, xu_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    ty_trans_submeso  (time, st_ocean, yu_ocean, xt_ocean) float32 3GB dask.array<chunksize=(12, 50, 150, 180), meta=np.ndarray>
    time_bounds       (time, nv) timedelta64[ns] 2kB dask.array<chunksize=(12, 2), meta=np.ndarray>
Attributes:
    filename:   ocean_month.nc
    title:      ACCESS-ESM_CMIP6
    grid_type:  regular
    grid_tile:  N/A

In [26]:
ds["tx_trans_gm"]

<xarray.DataArray 'tx_trans_gm' (time: 120, st_ocean: 50, yt_ocean: 300,
                                 xu_ocean: 360)> Size: 3GB
dask.array<concatenate, shape=(120, 50, 300, 360), dtype=float32, chunksize=(12, 50, 150, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 2kB -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
  * time      (time) datetime64[ns] 960B 1980-01-14T12:00:00 ... 1989-12-14T1...
  * xu_ocean  (xu_ocean) float64 3kB -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * st_ocean  (st_ocean) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
Attributes:
    long_name:      T-cell mass i-transport from GM
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT

In [ ]:
# tx_trans_gm
try:
    print("Loading tx_trans_gm")
    tx_trans_gm = ds["tx_trans_gm"].chunk({'time':-1})
    print("\ntx_trans_gm: ", tx_trans_gm)
    print("Saving tx_trans_gm to: ", f'{outputdir}/month_tx_trans_gm_{decade}s.nc')
    tx_trans_gm.to_netcdf(f'{outputdir}/month_tx_trans_gm_{decade}s.nc', compute=True)
except Exception:
    print(f'Error processing {model} {member} tx_trans_gm')
    print(traceback.format_exc())

Loading tx_trans_gm

tx_trans_gm:  <xarray.DataArray 'tx_trans_gm' (time: 120, st_ocean: 50, yt_ocean: 300,
                                 xu_ocean: 360)> Size: 3GB
dask.array<rechunk-merge, shape=(120, 50, 300, 360), dtype=float32, chunksize=(2, 50, 150, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 2kB -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
  * time      (time) datetime64[ns] 960B 1980-01-14T12:00:00 ... 1989-12-14T1...
  * xu_ocean  (xu_ocean) float64 3kB -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * st_ocean  (st_ocean) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
Attributes:
    long_name:      T-cell mass i-transport from GM
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
Saving tx_trans_gm to:  /g/data/xv83/TMIP/data/ACCESS-ESM1-5/HI-05/month_tx_trans_gm_1980s.nc


In [28]:
            # ty_trans_gm
            try:
                print("Loading ty_trans_gm")
                ty_trans_gm = ds["ty_trans_gm"]
                print("\nty_trans_gm: ", ty_trans_gm)
                print("Saving ty_trans_gm to: ", f'{outputdir}/month_ty_trans_gm_{decade}s.nc')
                ty_trans_gm.to_netcdf(f'{outputdir}/month_ty_trans_gm_{decade}s.nc', compute=True)
            except Exception:
                print(f'Error processing {model} {member} ty_trans_gm')
                print(traceback.format_exc())

Loading ty_trans_gm

ty_trans_gm:  <xarray.DataArray 'ty_trans_gm' (time: 120, st_ocean: 50, yu_ocean: 300,
                                 xt_ocean: 360)> Size: 3GB
dask.array<concatenate, shape=(120, 50, 300, 360), dtype=float32, chunksize=(12, 50, 150, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 3kB -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * time      (time) datetime64[ns] 960B 1980-01-14T12:00:00 ... 1989-12-14T1...
  * yu_ocean  (yu_ocean) float64 2kB -77.75 -77.51 -77.26 ... 89.1 89.55 90.0
  * st_ocean  (st_ocean) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
Attributes:
    long_name:      T-cell mass j-transport from GM
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
Saving ty_trans_gm to:  /g/data/xv83/TMIP/data/ACCESS-ESM1-5/HI-05/month_ty_trans_gm_1980s.nc


In [29]:
            # tx_trans_submeso
            try:
                print("Loading tx_trans_submeso")
                tx_trans_submeso_var = ds["tx_trans_submeso"]
                print("\ntx_trans_submeso: ", tx_trans_submeso_var)
                print("Saving tx_trans_submeso to: ", f'{outputdir}/month_tx_trans_submeso_{decade}s.nc')
                tx_trans_submeso.to_netcdf(f'{outputdir}/month_tx_trans_submeso_{decade}s.nc', compute=True)
            except Exception:
                print(f'Error processing {model} {member} tx_trans_submeso')
                print(traceback.format_exc())

Loading tx_trans_submeso

tx_trans_submeso:  <xarray.DataArray 'tx_trans_submeso' (time: 120, st_ocean: 50, yt_ocean: 300,
                                      xu_ocean: 360)> Size: 3GB
dask.array<concatenate, shape=(120, 50, 300, 360), dtype=float32, chunksize=(12, 50, 150, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 2kB -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
  * time      (time) datetime64[ns] 960B 1980-01-14T12:00:00 ... 1989-12-14T1...
  * xu_ocean  (xu_ocean) float64 3kB -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * st_ocean  (st_ocean) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
Attributes:
    long_name:      T-cell mass i-transport from submesoscale param
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
Saving tx_trans_submeso to:  /g/data/xv83/TMIP/data/ACCESS-ESM1-5/HI-05/month_tx_trans_submeso_1980s.nc
Error processing ACCESS-ESM1-5 HI-

In [ ]:
            # ty_trans_submeso
            try:
                print("Loading ty_trans_submeso")
                ty_trans_submeso_var = ds["ty_trans_submeso"]
                print("\nty_trans_submeso: ", ty_trans_submeso_var)
                print("Saving ty_trans_submeso to: ", f'{outputdir}/month_ty_trans_submeso_{decade}s.nc')
                ty_trans_submeso.to_netcdf(f'{outputdir}/month_ty_trans_submeso_{decade}s.nc', compute=True)
            except Exception:
                print(f'Error processing {model} {member} ty_trans_submeso')
                print(traceback.format_exc())